## importation and routings

In [1]:
import sys
sys.path.append('/UltimeTradingBot/Crypto_backtest_tools')
%cd /UltimeTradingBot/Crypto_backtest_tools
from utilities.get_data import get_historical_from_db
from utilities.backtesting import basic_single_asset_backtest, plot_wallet_vs_asset, get_metrics
import pandas as pd
import ccxt
import matplotlib.pyplot as plt
#import ta
import numpy as np
import gc
import random
from sklearn.utils import shuffle
import seaborn as sns
import tensorflow as tf

from utilities.backtesting import plot_wallet_vs_asset, get_metrics, get_n_columns, basic_multi_asset_backtest, plot_sharpe_evolution, plot_bar_by_month
#from utilities.custom_indicators import SuperTrend
pd.options.mode.chained_assignment = None  # default='warn'
import gc
gc.collect()    

import warnings
warnings.filterwarnings('ignore')
PRERR=False
def prerr(err):
    if PRERR:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

PDEBUG=True
def pdebug(err):
    if PDEBUG:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

/UltimeTradingBot/Crypto_backtest_tools


2022-09-30 23:24:03.702998: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-30 23:24:03.899785: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Usefull Global and Config Vars

In [2]:
WINDOW_SIZE=10
window=WINDOW_SIZE
MAX_FORCAST_SIZE=3
BUY_PCT=0.35
SELL_PCT=0.2
DATA_DIR='/UltimeTradingBot/Data/'

### importing Data

In [3]:
Binance_USDT_HALAL = [
    "BTC/USDT",
    "LUNA/USDT",
    "ETH/USDT",
    "GMT/USDT",
    "UST/USDT",
    "SOL/USDT",
    "APE/USDT",
    "XRP/USDT",
    "IDEX/USDT",
    "AVAX/USDT",
    "DOT/USDT",
    "ADA/USDT",
    "JASMY/USDT",
    "TRX/USDT",
    "NEAR/USDT",
    "AXS/USDT",
    "GAL/USDT",
    "GALA/USDT",
    "SHIB/USDT",
    "ZIL/USDT",
    "ENS/USDT",
    "DOGE/USDT",
    "LTC/USDT",
    "EUR/USDT",
    "MANA/USDT",
    "DAR/USDT",
    "WAVES/USDT",
    "LAZIO/USDT",
    "ALICE/USDT",
    "ROSE/USDT",
    "ZEC/USDT",
    "ALGO/USDT",
    "GRT/USDT",
    "PSG/USDT",
    "SLP/USDT",
    "EOS/USDT",
    "PORTO/USDT",
    "ICP/USDT",
    "EGLD/USDT",
    "XMR/USDT",
    "KDA/USDT",
    "ETC/USDT",
    "MBOX/USDT",
    "OGN/USDT",
    "AR/USDT",
    "GLMR/USDT",
    "LOKA/USDT",
    "XLM/USDT",
    "MTL/USDT",
    "SNX/USDT",
    "PYR/USDT",
    "DASH/USDT",
    "CITY/USDT",
    "ASTR/USDT",
    "IOTA/USDT",
    "ACM/USDT",
    "BAR/USDT",
    "JUV/USDT",
    "SYS/USDT",
    "RVN/USDT",
    "MBL/USDT",
    "REN/USDT",
    "JST/USDT",
    "OMG/USDT",
    "ATM/USDT",
    "XEC/USDT",
    "STORJ/USDT",
    "ZRX/USDT",
    "SRM/USDT",
    "ICX/USDT",
    "API3/USDT",
    "ONT/USDT",
    "SKL/USDT",
    "MULTI/USDT",
    "QTUM/USDT",
    "COCOS/USDT",
    "VOXEL/USDT",
    "HIVE/USDT",
    "KP3R/USDT",
    "ATA/USDT",
    "STMX/USDT",
    "ADX/USDT",
    "HIGH/USDT",
    "NULS/USDT",
    "MLN/USDT",
    "YGG/USDT",
    "SC/USDT",
    "CKB/USDT",
    "TOMO/USDT",
    "STX/USDT",
    "FLUX/USDT",
    "DNT/USDT",
    "ORN/USDT",
    "PLA/USDT",
    "BADGER/USDT",
    "DF/USDT",
    "MOB/USDT",
    "LPT/USDT",
    "SCRT/USDT",
    "RAD/USDT",
    "NMR/USDT",
    "ELF/USDT",
    "TORN/USDT",
    "T/USDT",
    "QUICK/USDT",
    "LSK/USDT",
    "FIDA/USDT",
    "XNO/USDT",
    "BTG/USDT",
    "GHST/USDT",
    "EPS/USDT"
]

pair_list = Binance_USDT_HALAL
tf = '1m'
oldest_pair = "BTC/USDT"
df_list1m = {}
df_list1d = {}
df_list1h = {}
df_list5m = {}
df_list15m = {}


for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1m', path="./database/")
    df_list1m[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1d', path="./database/")
    df_list1d[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1h', path="./database/")
    df_list1h[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '5m', path="./database/")
    df_list5m[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(
        ccxt.binance(), pair, '15m', path="./database/")
    df_list15m[pair] = df.loc[:]
del(df)
df_list = df_list1m
prerr("Data load 100% use df_list1d[\"BTC/USDT\"] for exemple to access")


In [4]:
### Testing Raw Data

In [5]:
df_list1m["BTC/USDT"].iloc[0:10]

,open,high,low,close,volume
date,,,,,
2020-07-01 00:00:00,9138.08,9138.16,9126.66,9126.75,29.777507
2020-07-01 00:01:00,9126.75,9129.91,9116.83,9118.63,64.406983
2020-07-01 00:02:00,9117.93,9119.68,9100.00,9104.43,210.822875
2020-07-01 00:03:00,9104.44,9115.31,9104.28,9115.02,38.691824
2020-07-01 00:04:00,9114.62,9118.62,9112.90,9117.23,22.128671
2020-07-01 00:05:00,9117.85,9117.85,9104.21,9109.62,92.137016
2020-07-01 00:06:00,9109.62,9112.97,9102.82,9111.05,33.733341
2020-07-01 00:07:00,9111.05,9118.61,9111.04,9118.60,39.883448
2020-07-01 00:08:00,9118.61,9121.15,9116.05,9120.52,18.666122


### Validated Fonctions

In [4]:
'''
Binance First Candle Finders
Creslin

Get list of all IDs on binance
Returns the first candle / launch timestamp to the minute for each
'''
import urllib.request
import json
import ccxt

def all_ids():
    # load all markets from binance into a list
    id = 'binance'
    exchange_found = id in ccxt.exchanges
    if exchange_found:
        exchange = getattr(ccxt, id)({})
        markets = exchange.load_markets()
        tuples = list(ccxt.Exchange.keysort(markets).items())

        ids = []
        for (k, v) in tuples:
            ids.append(v['id'])

        return ids

def give_first_kline_open_stamp(interval, symbol, start_ts=1499990400000):
        '''
        Returns the first kline from an interval and start timestamp and symbol
        :param interval:  1w, 1d, 1m etc - the bar length to query
        :param symbol:    BTCUSDT or LTCBTC etc
        :param start_ts:  Timestamp in miliseconds to start the query from
        :return:          The first open candle timestamp
        '''

        url_stub = "http://api.binance.com/api/v1/klines?interval="

        #/api/v1/klines?interval=1m&startTime=1536349500000&symbol=ETCBNB
        addInterval   = url_stub     + str(interval) + "&"
        addStarttime  = addInterval   + "startTime="  + str(start_ts) + "&"
        addSymbol     = addStarttime + "symbol="     + str(symbol)
        url_to_get = addSymbol

        kline_data = urllib.request.urlopen(url_to_get).read().decode("utf-8")
        kline_data = json.loads(kline_data)

        return kline_data[0][0]


# Get list of all IDs on binance
def get_crypto_metadata(pair_list):
    Binance_USDT_HALAL=pair_list
    ids = []
    #ids = all_ids()
    for halalpair in Binance_USDT_HALAL:
    #    print( halalpair.replace('/',''))
        ids.append(halalpair.replace('/',''))
    #print(ids)
    MetaData=pd.DataFrame(ids)
    MetaData["Pair"]=Binance_USDT_HALAL
    counters=0
    for this_id in ids:
        '''
        Find launch Week of symbol, start at Binance launch date 2017-07-14 (1499990400000)
        Find launch Day of symbol in week
        Find launch minute of symbol in day
        '''

        symbol_launch_week_stamp   = give_first_kline_open_stamp('1w', this_id, 1499990400000 )
        symbol_launch_day_stamp    = give_first_kline_open_stamp('1d', this_id, symbol_launch_week_stamp)
        symbol_launch_minute_stamp = give_first_kline_open_stamp('1m', this_id, symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_week_stamp"]=str(symbol_launch_week_stamp)
        MetaData.loc[counters,"launch_day_stamp"]=str(symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_minute"]=pd.to_datetime(symbol_launch_minute_stamp, unit='ms')

        counters += 1

        #print("Week stamp", symbol_launch_week_stamp)
        #print("Day  stamp", symbol_launch_day_stamp)
        #print("Min  stamp", symbol_launch_minute_stamp)

        print(this_id, "launched", symbol_launch_minute_stamp )
    return MetaData
    #print("")

In [5]:
#MetaData=get_crypto_metadata(Binance_USDT_HALAL)
MetaData = pd.read_csv("../Data/MetaData.csv")
#allok = pd.read_csv('D:/+DATA+/allok_w15.csv')

In [6]:
#MetaData.to_csv("D:\+DATA+\MetaData.csv")
pair_list=Binance_USDT_HALAL
window=20
buy_weight=50
sample_size=10000
min_days=10
buffer_size=100000
MetaData

,Unnamed: 0,0,Pair,launch_week_stamp,launch_day_stamp,launch_minute
0,0,BTCUSDT,BTC/USDT,1502668800000,1502928000000,2017-08-17 04:00:00
1,1,LUNAUSDT,LUNA/USDT,1597622400000,1597968000000,2020-08-21 10:00:00
2,2,ETHUSDT,ETH/USDT,1502668800000,1502928000000,2017-08-17 04:00:00
3,3,GMTUSDT,GMT/USDT,1646611200000,1646784000000,2022-03-09 12:00:00
4,4,USTUSDT,UST/USDT,1639958400000,1640304000000,2021-12-24 08:00:00
...,...,...,...,...,...,...
106,106,FIDAUSDT,FIDA/USDT,1632700800000,1632960000000,2021-09-30 12:00:00
107,107,XNOUSDT,XNO/USDT,1642982400000,1643328000000,2022-01-28 08:00:00
108,108,BTGUSDT,BTG/USDT,1618185600000,1618531200000,2021-04-16 07:00:00
109,109,GHSTUSDT,GHST/USDT,1629072000000,1629417600000,2021-08-20 10:00:00


In [7]:
# btc1m["high1"]=btc1m.loc[1:]["high"]

def expand_row(dataframe, window=2):
    df = dataframe.copy()
    for i in range(1, window+1):
        df["high"+str(i)] = df["high"][i:]
        df["low"+str(i)] = df["low"][i:]
        df["open"+str(i)] = df["open"][i:]
        df["close"+str(i)] = df["close"][i:]
        df["volume"+str(i)] = df["volume"][i:]
    return df

def expand_previous(dataframe, window=10):
    df = dataframe.copy()
    if window >= len(df):
        for i in range(1, window+1):
            df.loc[window:len(df),"high-"+str(i)]=None
            df.loc[window:len(df),"low-"+str(i)]=None
            df.loc[window:len(df),"open-"+str(i)]=None            
            df.loc[window:len(df),"close-"+str(i)]=None            
            df.loc[window:len(df),"volume-"+str(i)]=None
        window=len(df)

    for i in range(1, window+1):
        try:
            df.loc[window:len(df),"high-"+str(i)]=None
            df["high-"+str(i)].iloc[window:len(df)]=df["high"][window-i:len(df)-i]

            df.loc[window:len(df),"low-"+str(i)]=None
            df["low-"+str(i)].iloc[window:len(df)]=df["low"][window-i:len(df)-i]

            df.loc[window:len(df),"open-"+str(i)]=None
            df["open-"+str(i)].iloc[window:len(df)]=df["open"][window-i:len(df)-i]            
            
            df.loc[window:len(df),"close-"+str(i)]=None
            df["close-"+str(i)].iloc[window:len(df)]=df["close"][window-i:len(df)-i]            
            
            df.loc[window:len(df),"volume-"+str(i)]=None
            df["volume-"+str(i)].iloc[window:len(df)]=df["volume"][window-i:len(df)-i]
            
            # df["high-"+str(i)][i:] = df["high"][i-1:]
            # df["low-"+str(i)][i:] = df["low"][i-1:]
            # df["open-"+str(i)][i:] = df["open"][i-1:]
            # df["close-"+str(i)][i:] = df["close"][i-1:]
            # df["volume-"+str(i)][i:] = df["volume"][i-1:]
        except:
            prerr("Error in     expand_previous: " +str(i))
    if window >= len(df): return df       
    return df.iloc[window:]


def expand_previous_err(dataframe, window=10):
    df = dataframe.copy()
    if window >= len(df):
        for i in range(1, window+1):
            df.loc[window:len(df),"high-"+str(i)]=None
            df.loc[window:len(df),"low-"+str(i)]=None
            df.loc[window:len(df),"open-"+str(i)]=None            
            df.loc[window:len(df),"close-"+str(i)]=None            
            df.loc[window:len(df),"volume-"+str(i)]=None
        window=len(df)

    for i in range(1, window+1):
            df.loc[window:len(df),"high-"+str(i)]=None
            df["high-"+str(i)].iloc[window:len(df)]=df["high"][window-i:len(df)-i]

            df.loc[window:len(df),"low-"+str(i)]=None
            df["low-"+str(i)].iloc[window:len(df)]=df["low"][window-i:len(df)-i]

            df.loc[window:len(df),"open-"+str(i)]=None
            df["open-"+str(i)].iloc[window:len(df)]=df["open"][window-i:len(df)-i]            
            
            df.loc[window:len(df),"close-"+str(i)]=None
            df["close-"+str(i)].iloc[window:len(df)]=df["close"][window-i:len(df)-i]            
            
            df.loc[window:len(df),"volume-"+str(i)]=None
            df["volume-"+str(i)].iloc[window:len(df)]=df["volume"][window-i:len(df)-i]
            
            # df["high-"+str(i)][i:] = df["high"][i-1:]
            # df["low-"+str(i)][i:] = df["low"][i-1:]
            # df["open-"+str(i)][i:] = df["open"][i-1:]
            # df["close-"+str(i)][i:] = df["close"][i-1:]
            # df["volume-"+str(i)][i:] = df["volume"][i-1:]
    if window >= len(df):
        return df
    return df.iloc[window:]

def expand_timeframe(df_minutes,df_hours, window=2):
    dfm = df_minutes.copy()
    for j in range(1, window+1):
        for i in df_hours[dfm.iloc[0].name:].index:
        #prerr(str(i))
            try:
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"high_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['high']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"low_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['low']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"open_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['open']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"close_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['close']
            except:
                prerr("Error Merging: "+str(i))
    
    return dfm


def float_or_not(var):
    try:
        x=float(var)
    except:
        x=None
    return x

def expand_to_1h(df_1m,df_1h, window=2):
    dfm = df_1m.copy()
    index_start=df_1h.index.intersection(dfm.index.round(freq='H'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=60, freq="min"))
                dfm.loc[timefragment,"high_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['high'])
                dfm.loc[timefragment,"low_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['low'])
                dfm.loc[timefragment,"open_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['open'])
                dfm.loc[timefragment,"close_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def expand_to_4h(df_1m,df_4h, window=2):
    dfm = df_1m.copy()
    #index_start=df_1h[str(dfm.iloc[0].name.round(freq='H')):].index.intersection(dfm.index)
    index_start=df_4h.index.intersection(dfm.index.round(freq='4H'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=4*60, freq="min"))
                dfm.loc[timefragment,"high_4h-"+str(j)]=float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['high'])
                dfm.loc[timefragment,"low_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['low'])
                dfm.loc[timefragment,"open_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['open'])
                dfm.loc[timefragment,"close_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def expand_to_1d(df_1m,df_1d, window=2,time_suffix="1d"):
    dfm = df_1m.copy()
    index_start=df_1d.index.intersection(dfm.index.round(freq='1d'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                prerr(i)
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=24*60, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def expand_to_5m(df_1m,df_5m, window=2,time_suffix="5m"):
    dfm = df_1m.copy()
    index_start=df_5m.index.intersection(dfm.index.round(freq='5 min'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=5, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm


def expand_to_15m(df_1m,df_15m, window=2,time_suffix="15m"):
    dfm = df_1m.copy()
    index_start=df_15m.index.intersection(dfm.index.round(freq='5 min'))
    for i in index_start:
        for j in range(1, window+1):    
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=15, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def rapid1d_expand(df1m,df1d,window=2):
    d1min=df1m.copy()
    d1day=df1d.loc[
    d1min.index[0].round(freq='1d'):
    d1min.index[len(d1min)-1].round(freq='1d')+pd.Timedelta('1 day')
    ].copy()
    d1day_pre=expand_previous(d1day,window)
    d1day_pre=d1day_pre.drop(columns=['open', 'low','close','high','volume'])
    d1day_pre=d1day_pre.add_suffix("_day")
    d1min=pd.merge_asof(
    d1min, d1day_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_day'),
    tolerance=pd.Timedelta('1 day'), allow_exact_matches=True, direction='backward')
    return d1min

def rapid1h_expand(df1m,df1h,window=2):
    d1min=df1m.copy()
    d1hour=df1h.loc[
    d1min.index[0].round(freq='H'):
    d1min.index[len(d1min)-1].round(freq='H')+pd.Timedelta('1 hour')
    ].copy()
    d1hour_pre=expand_previous(d1hour,window)
    d1hour_pre=d1hour_pre.drop(columns=['open', 'low','close','high','volume'])
    d1hour_pre=d1hour_pre.add_suffix("_hour")
    d1min=pd.merge_asof(
    d1min, d1hour_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_hour'),
    tolerance=pd.Timedelta('1 hour'), allow_exact_matches=True, direction='backward')
    return d1min


def rapid5m_expand(df1m,df5m,window=2):
    d1min=df1m.copy()
    d5min=df5m.loc[
    d1min.index[0].round(freq='5 min'):
    d1min.index[len(d1min)-1].round(freq='5 min')+pd.Timedelta('5 min')
    ].copy()
    d5min_pre=expand_previous(d5min,window)
    d5min_pre=d5min_pre.drop(columns=['open', 'low','close','high','volume'])
    d5min_pre=d5min_pre.add_suffix("_5min")
    d1min=pd.merge_asof(
    d1min, d5min_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_5min'),
    tolerance=pd.Timedelta('5 min'), allow_exact_matches=True, direction='backward')
    return d1min

def rapid15m_expand(df1m,df15m,window=2):
    d1min=df1m.copy()
    d15min=df15m.loc[
    d1min.index[0].round(freq='15 min'):
    d1min.index[len(d1min)-1].round(freq='15 min')+pd.Timedelta('15 min')
    ].copy()
    d15min_pre=expand_previous(d15min,window)
    d15min_pre=d15min_pre.drop(columns=['open', 'low','close','high','volume'])
    d15min_pre=d15min_pre.add_suffix("_15min")
    d1min=pd.merge_asof(
    d1min, d15min_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_15min'),
    tolerance=pd.Timedelta('15 min'), allow_exact_matches=True, direction='backward')
    return d1min

def full_expand(df1m,df5m,df15m,df1h,df1d,window=10):
    d1min=df1m.copy()
    d1min=expand_previous(d1min,window=window)
    d1min=rapid1d_expand(d1min,df1d,window)
    d1min=rapid1h_expand(d1min,df1h,window)
    d1min=rapid15m_expand(d1min,df15m,window)
    d1min=rapid5m_expand(d1min,df5m,window)
    return d1min

def day_expand(data_full):
    ser = pd.to_datetime(pd.Series(data_full.index))
    data_full["day"]=ser.dt.isocalendar().day.values
    data_full["hour"]=ser.dt.hour.values
    data_full["minute"]=ser.dt.minute.values

# merging
def pair_btc(pair="LTC/USDT",window=2):
    Pair_Full=full_expand(df_list1m[pair],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[df_list1m[pair].iloc[0].name:
        df_list1m[pair].iloc[len(df_list1m[pair])-1].name],
        df_list5m["BTC/USDT"],df_list15m["BTC/USDT"],df_list1h["BTC/USDT"],df_list1d["BTC/USDT"],window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='outer',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    return Merged






In [8]:

def buy_results(df,min_pct=BUY_PCT):
    mino=min_pct*0.01
    df["buy"]=(
        ((df["high"].shift(periods=1, freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| ((
          df["high"].shift(periods=2, freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| ((
          df["high"].shift(periods=3, freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)
    ).replace({False: 0, True: 1}) 
 
def buy_results_gen(df,min_pct=BUY_PCT,window=3):
    mino=min_pct*0.01
    codep1='df["buy"]=((('
    for i in range(1,window):
        codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
    codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
    code=codep1+codep2
    print(code)
    exec(code)

def buy_sell(df,BUY_PCT=BUY_PCT,SELL_PCT=SELL_PCT,window=3):
    mino=BUY_PCT*0.01
    maxo=-SELL_PCT*0.01
    codep1='df["buy"]=((('
    for i in range(1,window):
        codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
    codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
    code=codep1+codep2
    prerr(code)
    exec(code)
    codep1='df["sell"]=((df["buy"]==0)&(( '
    for i in range(1,window):
        codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )& (('
    codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )).replace({False: 0, True: 1})'
    code=codep1+codep2
    prerr(code)
    exec(code)
    df["bs"]=((df['buy']==1 )& (df['sell']==0)).replace({False: 0, True: 1})

In [9]:


def Meta_expand(data_full,metadt,pair):
    data_full["lunch_day"]=int(-(pd.to_datetime(metadt[metadt["Pair"] == pair]["launch_minute"])-pd.Timestamp('2020-01-01 00:00:00.000000')).dt.days)

def mini_expand(pair="LTC/USDT",i=0,j=10000,window=2):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[Pair_Full.iloc[0].name-pd.Timedelta(str(window-1) +" min"):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],
        df_list15m["BTC/USDT"],
        df_list1h["BTC/USDT"],
        df_list1d["BTC/USDT"],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    return Merged

def mini_expand2(pair="LTC/USDT",i=0,j=10000,window=2,metadata=MetaData):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[Pair_Full.iloc[0].name-pd.Timedelta(str(window-1) +" min"):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],
        df_list15m["BTC/USDT"],
        df_list1h["BTC/USDT"],
        df_list1d["BTC/USDT"],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    Meta_expand(Merged,metadata,pair)
    buy_sell(Merged,BUY_PCT=BUY_PCT,SELL_PCT=SELL_PCT,window=7)
    return Merged

def mini_expand3(pair="LTC/USDT",i=0,j=10000,window=2,metadata=MetaData,high_weight=3):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[Pair_Full.iloc[0].name-pd.Timedelta(str(window-1) +" min"):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],
        df_list15m["BTC/USDT"],
        df_list1h["BTC/USDT"],
        df_list1d["BTC/USDT"],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    Meta_expand(Merged,metadata,pair)
    buy_sell(Merged,BUY_PCT=BUY_PCT,SELL_PCT=SELL_PCT,window=MAX_FORCAST_SIZE)
    Merged["high"]=(Merged["open"]+high_weight*Merged["high"]+Merged["low"]+Merged["close"])/(3+high_weight)
    Merged.rename(columns={"high":"price"},inplace = True)
    Merged["BTC_high"]=(Merged["BTC_open"]+high_weight*Merged["BTC_high"]+Merged["BTC_low"]+Merged["BTC_close"])/(3+high_weight)
    Merged.rename(columns={"BTC_high":"BTC_price"},inplace = True)
    Merged=Merged.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])
    for key in Merged.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["BTC_price"]-Merged[key])/Merged["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["price"]-Merged[key])/Merged["price"]
    return Merged



def slow_expand(pair="LTC/USDT",i=0,j=100000,window=3):
    df=mini_expand(pair=pair,i=i,j=j,window=window)
    for mx in range(1,int(len(df_list1m[pair])/j)+1) :
        df=pd.concat([df,
        mini_expand(pair=pair,
        i=(mx*j)-window,
        j=(mx+1)*j,
        window=window)],axis=0)
    return df

def pair_data_gen(pair="LTC/USDT",i=0,j=100000,window=3,metadata=MetaData):
    df=mini_expand3(pair=pair,i=i,j=j,window=window,metadata=metadata)
    for mx in range(1,int(len(df_list1m[pair])/j)+1) :
        df=pd.concat([df,
        mini_expand3(pair=pair,
        i=(mx*j)-window,
        j=(mx+1)*j,
        window=window,metadata=metadata)],axis=0)
        # Meta_expand(df,metadata,pair)
        # buy_sell(df,BUY_PCT=BUY_PCT,SELL_PCT=SELL_PCT,window=7)
        #print("loop "+str(mx)+"--> size of df: "+str(len(df)))
    return df

###

def data_is_enough(df,days=10,window=10):
    if days <= window:
        return df[~df.isnull().any(axis=1) |(df["open-"+str(days)+"_day"].isnull() & ~df["open-"+str(window-1)+"_hour"].isnull()  & ~df["open-"+str(window-1)+"_5min"].isnull() & ~df["open-"+str(days-1)+"_day"].isnull())]
    else:
        prerr("number of days must be equal or lower than window")
        return df

def data_cleanup(df):
    return df[~df.isnull().any(axis=1)]
    

def data_shufler(df):
    x = len(df)
    df["num_index"] = range(0, x, 1)
    df.set_index(df['num_index'], inplace=True)
    df = df.reindex(np.random.permutation(df.index))
    df= df.drop("num_index",axis=1)
    df = df.reindex(np.random.permutation(df.index))
    return df
    
def data_np_shufler(df):
    df = shuffle(df)
    #df = df.reindex(np.random.permutation(df.index))
        



def data_chooser(df,weight=50,row_numbers=100000):
    df=data_shufler(df)
    if row_numbers>=len(df):
        row_numbers=len(df)
    df=pd.concat([df[df["buy"]==1].iloc[:int(row_numbers*weight*0.01)],
                 df[df["buy"]==0].iloc[:int(row_numbers*(100-weight)*0.01)]])
    df=data_shufler(df)
    return df
    

def data_looper(pair_list=Binance_USDT_HALAL,window=15,buy_weight=50,sample_size=100000,min_days=10,buffer_size=100000):
    xdf=pd.DataFrame()
    for pair in pair_list:
        if pair != "BTC/USDT":
            print("working on: "+pair)
            df=pair_data_gen(pair=pair,i=0,j=buffer_size,window=window)
            gc.collect()

            df=data_is_enough(df,days=min_days,window=window)
            gc.collect()

            df=data_chooser(df,weight=buy_weight,row_numbers=sample_size)
            gc.collect()

            print(pair+" is processed")
            xdf=pd.concat([xdf,df],axis=0)
            del(df)
            gc.collect()
        else:
            print("ignore BTC")
    return xdf

def volume_cleaner(df):
    VolRemover=["volume","volume-1","BTC_volume","BTC_volume-1"]
    for key in df.keys():
        if key.find("volume-1_") != -1 :
            VolRemover.append(key)
    df=df.drop(columns=VolRemover)
    return df


In [12]:
#test=mini_expand3(pair="LTC/USDT",i=0,j=10000,window=100,metadata=MetaData)

In [13]:
#df[df.isnull().any(axis=1)]


In [28]:
testdt=mini_expand3(pair="GMT/USDT",i=100000,j=101000,window=10,metadata=MetaData,high_weight=0)[110:188]
testdt.to_csv("livetest.csv")
testdt

,price,volume,high-1,low-1,open-1,close-1,volume-1,high-2,low-2,open-2,...,BTC_open-10_5min,BTC_close-10_5min,BTC_volume-10_5min,day,hour,minute,lunch_day,buy,sell,bs
date,,,,,,,,,,,,,,,,,,,,,
2022-05-18 00:40:00,1.609883,95158.6,-0.002296,0.001164,-0.002296,0.001133,95158.6,-0.002296,0.001164,-0.002296,...,-0.000163,0.000681,92.39277,3,0,40,-798,0,0,0
2022-05-18 00:41:00,1.606627,81512.5,-0.001602,0.001380,-0.000867,-0.000512,81512.5,-0.001602,0.001380,-0.000867,...,-0.000879,-0.000034,92.39277,3,0,41,-798,1,0,1
2022-05-18 00:42:00,1.606737,94961.6,-0.001259,0.000969,-0.000158,-0.000811,94961.6,-0.001259,0.000969,-0.000158,...,-0.001169,-0.000324,92.39277,3,0,42,-798,0,0,0
2022-05-18 00:43:00,1.605610,102584.1,-0.001707,0.001003,-0.001569,0.000567,102584.1,-0.001707,0.001003,-0.001569,...,-0.001151,-0.000306,92.39277,3,0,43,-798,0,0,0
2022-05-18 00:44:00,1.605063,108532.8,-0.001568,0.000276,0.000276,-0.000552,108532.8,-0.001568,0.000276,0.000276,...,-0.001075,-0.000230,92.39277,3,0,44,-798,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-18 01:53:00,1.579340,301338.5,-0.005604,0.003046,0.000760,-0.003805,301338.5,-0.005604,0.003046,0.000760,...,-0.002560,-0.000579,281.35293,3,1,53,-798,0,1,0
2022-05-18 01:54:00,1.584980,398427.2,-0.002883,0.002158,-0.000032,-0.002126,398427.2,-0.002883,0.002158,-0.000032,...,-0.002207,-0.000226,281.35293,3,1,54,-798,0,0,0
2022-05-18 01:55:00,1.587250,206516.8,-0.001336,0.001046,-0.000479,-0.000567,206516.8,-0.001336,0.001046,-0.000479,...,-0.000775,0.001916,416.35772,3,1,55,-798,0,0,0


In [29]:
testdt[testdt["bs"]==1]

,price,volume,high-1,low-1,open-1,close-1,volume-1,high-2,low-2,open-2,...,BTC_open-10_5min,BTC_close-10_5min,BTC_volume-10_5min,day,hour,minute,lunch_day,buy,sell,bs
date,,,,,,,,,,,,,,,,,,,,,
2022-05-18 00:41:00,1.606627,81512.5,-0.001602,0.001380,-0.000867,-0.000512,81512.5,-0.001602,0.001380,-0.000867,...,-0.000879,-0.000034,92.39277,3,0,41,-798,1,0,1
2022-05-18 00:46:00,1.589020,376281.0,-0.005670,0.002756,-0.001724,-0.001032,376281.0,-0.005670,0.002756,-0.001724,...,-0.000322,0.000858,118.00264,3,0,46,-798,1,0,1
2022-05-18 00:47:00,1.587743,193605.9,-0.002227,0.001287,-0.001969,0.000682,193605.9,-0.002227,0.001287,-0.001969,...,-0.000970,0.000211,118.00264,3,0,47,-798,1,0,1
2022-05-18 00:48:00,1.586560,463553.9,-0.003498,0.000876,-0.000082,-0.000794,463553.9,-0.003498,0.000876,-0.000082,...,-0.001753,-0.000571,118.00264,3,0,48,-798,1,0,1
2022-05-18 00:49:00,1.587120,219995.4,-0.002514,0.000737,-0.000510,-0.000227,219995.4,-0.002514,0.000737,-0.000510,...,-0.001693,-0.000512,118.00264,3,0,49,-798,1,0,1
2022-05-18 00:53:00,1.579803,306786.4,-0.002118,0.001699,-0.001574,-0.000124,306786.4,-0.002118,0.001699,-0.001574,...,-0.000434,0.000727,81.74724,3,0,53,-798,1,0,1
2022-05-18 00:54:00,1.574103,688920.6,-0.004439,0.003102,-0.003568,0.000466,688920.6,-0.004439,0.003102,-0.003568,...,-0.001027,0.000134,81.74724,3,0,54,-798,1,0,1
2022-05-18 00:55:00,1.573160,170951.6,-0.002346,0.001246,-0.000133,-0.001112,170951.6,-0.002346,0.001246,-0.000133,...,0.000340,-0.000315,63.81183,3,0,55,-798,1,0,1
2022-05-18 00:56:00,1.573950,147251.9,-0.001614,0.000832,-0.000585,-0.000248,147251.9,-0.001614,0.000832,-0.000585,...,0.001396,0.000741,63.81183,3,0,56,-798,1,0,1


# Start Data genration Process

In [ ]:
import gc
ww=WINDOW_SIZE
gc.collect()
df=data_looper(pair_list=Binance_USDT_HALAL,window=ww,buy_weight=50,sample_size=10000,min_days=3,buffer_size=100000)
gc.collect()
df=volume_cleaner(df)
#df=data_shufler(df)
#df=df.reindex(np.random.permutation(df.index))
#df=df.reindex(np.random.permutation(df.index))
gc.collect()
df["num_index2"] = range(0, len(df), 1)
df.set_index(df['num_index2'], inplace=True)
gc.collect()
df = df.reindex(np.random.permutation(df.index))
gc.collect()
df= df.drop("num_index2",axis=1)
try:
    df= df.drop("num_index",axis=1)
except:
    print("no numindex ")
gc.collect()

df = df.reindex(np.random.permutation(df.index))
#df = df.reindex(np.random.permutation(df.index))
df["index"] = range(0, len(df), 1)
df.set_index(df['index'], inplace=True)
try:
    df= df.drop("num_index2",axis=1)
except:
    print("no numindex 2")
gc.collect()

try:
    df= df.drop("index",axis=1)
except:
    print("index")
gc.collect()

try:
    df= df.drop(["num_index2","index"],axis=1,index=False)
except:
    print("no numindex 2")
df.to_csv(DATA_DIR+str(ww)+'_CryIn_NoVol.csv')


In [ ]:
df= df.drop("index",axis=1)

In [ ]:
#window=20
#pair="GAL/USDT"
#df=mini_expand("LTC/USDT",i=0,j=100000,window=20)
#df=full_expand(df_list1m[pair].iloc[0:10000],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window=20)
#df=pair_data_gen(pair,i=0,j=100000,window=20,metadata=MetaData)
#df=data_is_enough(df,days=min_days,window=window)
#df=data_chooser(df,weight=20,row_numbers=100000)

#df = expand_previous_err(df_list1d[pair],20)


In [ ]:
#df_list1d[pair]
#df["high-1"].iloc[window:len(df)]=df["high"][window-i:len(df)-i]

In [ ]:
#df = df.reindex(np.random.permutation(df.index))
#df=df.drop("num_index",axis=1)

In [ ]:
#df["date"]

In [ ]:
#df.iloc[100000]["open-9_hour"]

In [ ]:
#df[df.out  df.isnull().any(axis=1)]

In [ ]:
#df[df.isnull().any(axis=1) & (df["buy"]==1)]
#df[(df["buy"]==1)]


In [ ]:
#ltc1m = expand_row(df_list1m["LTC/USDT"], 3)
#sys.getsizeof(allok)/(1024**2)

In [ ]:
# df.to_excel("D:\+DATA+\LTCtest.xlsx", sheet_name='LTC', na_rep='', float_format=None,
#  columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, 
# inf_rep='inf', verbose=True, freeze_panes=None, storage_options=None)
#df.to_csv("D:\+DATA+\LTCtest.csv")

In [ ]:
#b=ccxt.binance()
#b.set_sandbox_mode(enabled=True)

In [ ]:
#b.api

In [ ]:
#Meta_expand(df,MetaData,pair)

In [ ]:
df

# part2


# Training

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
import os
pd.options.mode.chained_assignment = None  # default='warn'

import gc


import tensorflow as tf


gc.collect()
df=df.reindex(np.random.permutation(df.index))
gc.collect()
df=df.reindex(np.random.permutation(df.index))
gc.collect()
df=df.reindex(np.random.permutation(df.index))
gc.collect()
sys.getsizeof(df)/(1024**2)
df


In [ ]:

for k in df.keys():print(k,end="  ;")
gc.collect()
# starting numpy process
## convert Pandas DataFrame to numpy object


In [ ]:

dt = df.drop(columns=["sell"]).to_numpy()
dt
gc.collect()
#del(df)
gc.collect()
gc.collect()


In [ ]:
dt[0,0]

In [ ]:

dt.shape
dt[142110,-1]
### Shuffle the rows Again
np.random.shuffle(dt)
sys.getsizeof(dt)/(1024**2)
#np.savetxt('np_shuffled_cryptodata_w15.csv', dt ,delimiter=',')


#dt=np.genfromtxt('np_shuffled_cryptodata_w15.csv', delimiter=',')
index_20pct= int(0.3*len(dt[:,0]))
print(index_20pct)
XVALIDATION= dt[:index_20pct, :-1]
YVALIDATION= dt[:index_20pct,-1]

XTRAIN= dt[index_20pct:, 0:-1]
YTRAIN= dt[index_20pct:,-1]
## Normalization
plt.hist(XTRAIN[:,0])
plt.ylabel("open_1min")
plt.show()
plt.hist(YTRAIN)
plt.ylabel("Output labels")
plt.show()

plt.hist(YVALIDATION)
plt.ylabel("Output labels")
plt.show()
Tenderalization (mean normalization)
mean = XTRAIN.mean(axis=0)
XTRAIN -= mean 
std = XTRAIN.std(axis=0)
XTRAIN /= std

XVALIDATION -=mean
XVALIDATION /= std
Normalization={"mean":mean.tolist(),"std":std.tolist()}
with open(Normalization_File, 'w') as fp:
            json.dump(Normalization, fp,  indent=4)
plt.hist(XTRAIN[:,0])
plt.ylabel("open column")
plt.show()
print(XTRAIN.shape)
print(YTRAIN.shape)
print(XVALIDATION.shape)
print(YVALIDATION.shape)
# Neural network model
### Model Generator
IN_DIM=len(XTRAIN[0,:])

#code genrator
global All_Hist
try:
    with open('Results_history.json') as json_file:
        All_Hist = json.load(json_file)
except:
    All_Hist={0.000001:"code"}

try:
    All_Hist.pop('1e-06')
except:
    print(All_Hist)

The neural network architecture:
- layer 1 : 764 neurons
- layer 2 : 8 neurons
- layer 3 : neurons 
## Model Test
## Compile the model 
model = Sequential()
model.add(Dense(int(IN_DIM/2),input_dim=IN_DIM,activation='softplus'))
model.add(Dense(int(IN_DIM/2),activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(int(IN_DIM/5),activation='softplus'))
model.add(Dense(int(IN_DIM/4),activation='softmax'))
model.add(Dense(int(IN_DIM/1),activation='softplus'))
model.add(Dense(1,activation='sigmoid'))
print(model.summary())
#model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
callbacks_a = ModelCheckpoint(filepath = Model_FileName,monitor ='val_accuracy',save_best_only = True, save_weights = True)
callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=20,verbose=1)
history = model.fit(XTRAIN,
                YTRAIN,
                validation_data=(XVALIDATION,YVALIDATION),
                epochs=1200,
                batch_size=5000,
                callbacks=[callbacks_a,callbacks_b])

print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")

print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")

accuracy = model.evaluate(XVALIDATION, YVALIDATION)

dt[0,740:-1]
### Use the model

# make probability predictions with the model
predictions = model.predict(XVALIDATION)
# round predictions 
rounded = [round(x[0]) for x in predictions]
rounded[121201]
YVALIDATION[121201]